In [ ]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("Replace").getOrCreate()

In [ ]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]
schema = ["employee_name","department","state","salary","age","bonus"]
simple_df = spark.createDataFrame(simpleData,  schema)
simple_df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [ ]:
#We need to Group by Department And State
#Output- Sum of salary and bonus
from pyspark.sql.functions import col
salaryempSalary = simple_df.groupBy('department','state').agg({'salary':'sum','bonus':'sum'})
salaryempSalary.withColumn('Total Sum',col('sum(bonus)')+col('sum(salary)')).show()

+----------+-----+----------+-----------+---------+
|department|state|sum(bonus)|sum(salary)|Total Sum|
+----------+-----+----------+-----------+---------+
|   Finance|   NY|     34000|     162000|   196000|
| Marketing|   NY|     21000|      91000|   112000|
|     Sales|   CA|     23000|      81000|   104000|
| Marketing|   CA|     18000|      80000|    98000|
|   Finance|   CA|     47000|     189000|   236000|
|     Sales|   NY|     30000|     176000|   206000|
+----------+-----+----------+-----------+---------+



In [ ]:
#Task 3- Group by department and Print the sum of salary, average of salary, minimum salary and maximim salary
from pyspark.sql.functions import sum,avg,min,max
final_df = simple_df.groupBy('department').agg(sum("salary").alias("totalsalary"), avg("salary").alias("avgSalary"),min("salary").alias("minsalary"),max("salary").alias("maxSalary"))


In [ ]:
final_df.show()

+----------+-----------+-----------------+---------+---------+
|department|totalsalary|        avgSalary|minsalary|maxSalary|
+----------+-----------+-----------------+---------+---------+
|     Sales|     257000|85666.66666666667|    81000|    90000|
|   Finance|     351000|          87750.0|    79000|    99000|
| Marketing|     171000|          85500.0|    80000|    91000|
+----------+-----------+-----------------+---------+---------+



In [ ]:
#Taking the files from web
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)

In [ ]:
#reading the file to create the dataframe
df = spark.read.csv("file://"+SparkFiles.get("iris.data"), inferSchema= True)

In [ ]:
#priting the column and showing the information
df.printSchema()
df.show()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.4|3.7|1.5|0.2|Iris-setosa|
|4.8|3.4|1.6|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.3|3.0|1.1|0.1|Iris-setosa|
|5.8|4.0|1.2|0.2|Iris-setosa|
|5.7|4.4|1.5|0.4|Iris-setosa|
|5.4|3.9|1.3|0.4|Iris-setosa|
|5.1|3.5|1.4|0.3|Iris-setosa|
|5.7|3.8|1.7|0.3|Iris-setosa|
|5.1|3.8|1.5|0.3|Iris-setosa|
+---+---+---+---+-----------+
only showing top 20 rows



In [ ]:
#renaming the column name
iris_df = df.withColumnRenamed('_c0', 'sepal_length') \
                    .withColumnRenamed('_c1', 'sepal_width') \
                    .withColumnRenamed('_c2', 'petal_length') \
                    .withColumnRenamed('_c3', 'petal_width') \
                    .withColumnRenamed('_c4', 'species')
iris_df.show()

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    species|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

In [ ]:
max_sepal_width = iris_df.agg(max("sepal_width")).collect()[0][0]
print("Maximum sepal width", max_sepal_width)

Maximum sepal width 4.4


In [ ]:
max_petal_width = iris_df.agg(max("petal_width")).collect()[0][0]
print("Maximum sepal width", max_petal_width)

Maximum sepal width 2.5


In [ ]:
max_sepal_length = iris_df.agg(max("sepal_length")).collect()[0][0]
print("Maximum sepal length", max_sepal_length)

Maximum sepal length 7.9


In [ ]:
max_petal_length = iris_df.agg(max("petal_length")).collect()[0][0]
print("Maximum sepal length", max_petal_length)

Maximum sepal length 6.9


In [ ]:
if max_sepal_width > max_petal_width:
    print("Sepal width has the maximum value",max_sepal_width)
else:
    print("Petal width has the maximum value",max_petal_width)

Sepal width has the maximum value 4.4


In [ ]:
if max_sepal_length > max_petal_length:
    print("Sepal length has the maximum value",max_sepal_length)
else:
    print("Petal length has the maximum value",max_petal_length)

Sepal length has the maximum value 7.9


In [ ]:
#creating a PySpark DataFrame from the data
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]
columns = ["first_name", "middle_name", "last_name", "dob", "gender", "salary"]
df = spark.createDataFrame(data, columns)

# showing the PySpark DataFrame
df.printSchema()
df.show()

root
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+----------+-----------+---------+----------+------+------+
|first_name|middle_name|last_name|       dob|gender|salary|
+----------+-----------+---------+----------+------+------+
|     James|           |    Smith|1991-04-01|     M|  3000|
|   Michael|       Rose|         |2000-05-19|     M|  4000|
|    Robert|           | Williams|1978-09-05|     M|  4000|
|     Maria|       Anne|    Jones|1967-12-01|     F|  4000|
|       Jen|       Mary|    Brown|1980-02-17|     F|    -1|
+----------+-----------+---------+----------+------+------+



In [ ]:
df = df.withColumn("salary", col("salary").cast("integer"))


In [ ]:
df.printSchema()
df.show()

root
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+----------+-----------+---------+----------+------+------+
|first_name|middle_name|last_name|       dob|gender|salary|
+----------+-----------+---------+----------+------+------+
|     James|           |    Smith|1991-04-01|     M|  3000|
|   Michael|       Rose|         |2000-05-19|     M|  4000|
|    Robert|           | Williams|1978-09-05|     M|  4000|
|     Maria|       Anne|    Jones|1967-12-01|     F|  4000|
|       Jen|       Mary|    Brown|1980-02-17|     F|    -1|
+----------+-----------+---------+----------+------+------+



In [ ]:
#Now create one column which as 5 times of your existing salary
df = df.withColumn("FiveTimessalary", col("salary") * 5)
df.show()


+----------+-----------+---------+----------+------+------+---------------+
|first_name|middle_name|last_name|       dob|gender|salary|FiveTimessalary|
+----------+-----------+---------+----------+------+------+---------------+
|     James|           |    Smith|1991-04-01|     M|  3000|          15000|
|   Michael|       Rose|         |2000-05-19|     M|  4000|          20000|
|    Robert|           | Williams|1978-09-05|     M|  4000|          20000|
|     Maria|       Anne|    Jones|1967-12-01|     F|  4000|          20000|
|       Jen|       Mary|    Brown|1980-02-17|     F|    -1|             -5|
+----------+-----------+---------+----------+------+------+---------------+



In [ ]:
#PYSpark SQL

In [ ]:
spark = SparkSession.builder.appName("Read Top 5 Rows from CSV").getOrCreate()
df = spark.read.csv("housing.csv")
df.show()

Py4JJavaError: ignored